In [28]:
import pandas as pd
import metnum
import numpy as np
import utils
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('../data/train.csv')

A = df[df['ciudad'] == 'Querétaro']
A

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
12,47890,hermosa casa en zibata queretaro 170m²,<p><strong>casa </strong>en venta en fracciona...,Casa,"Pitahayas 2 - 58, Zibata",Querétaro,Querétaro,0.0,3.0,2.0,...,83666.0,20.682878,-100.317503,2013-08-12 00:00:00,0.0,1.0,0.0,1.0,1.0,1500000.0
16,283945,preciosa casa en cumbres del lago \t,<p>clave: vcln2450 fecha de actualizaci&oacute...,Casa,"CUMBRES DEL LAGO JURIQUILLA, QUERETARO, MEXICO",Querétaro,Querétaro,0.0,3.0,2.0,...,84338.0,20.708760,-100.459303,2013-03-06 00:00:00,0.0,0.0,0.0,1.0,1.0,2450000.0
18,40421,últimos terrenos en preventa!! en bukara coto ...,"terreno ubicado en bukara coto club, el cual s...",Terreno,Milenio,Querétaro,Querétaro,NaN,NaN,0.0,...,339455.0,NaN,NaN,2016-12-03 00:00:00,0.0,0.0,1.0,1.0,1.0,794078.0
19,87655,NaN,-- rcv151104-lv-27 -- linda casa con area d...,Casa,SENDA CELESTIAL 10,Querétaro,Querétaro,0.0,3.0,2.0,...,339455.0,NaN,NaN,2016-01-15 00:00:00,0.0,0.0,0.0,0.0,0.0,1850000.0
21,224513,casa en venta en querétaro,nocnok id: mx15-bk0340. hermosa casa en conjun...,Casa en condominio,NaN,Querétaro,Querétaro,NaN,3.0,2.0,...,NaN,NaN,NaN,2016-02-01 00:00:00,0.0,0.0,0.0,0.0,0.0,2300000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239872,110687,departamento en zibata,<p>excelente departamento nuevo cuenta con 2 r...,Casa,NaN,Querétaro,Querétaro,NaN,2.0,2.0,...,NaN,NaN,NaN,2015-10-10 00:00:00,0.0,0.0,0.0,0.0,0.0,1480000.0
239913,133767,pedregal de schoensttant nueva con roof garden,pedregal de schoensttant \ncasa 3 niveles114...,Casa,NaN,Querétaro,Querétaro,NaN,NaN,NaN,...,84285.0,NaN,NaN,2015-11-15 00:00:00,0.0,0.0,0.0,0.0,0.0,1350000.0
239922,9268,"casa en venta 3 recámaras, privada, vigilancia...","casas en privada, con excelentes espacios, jar...",Casa en condominio,NaN,Querétaro,Querétaro,3.0,3.0,2.0,...,NaN,0.000000,0.000000,2016-06-03 00:00:00,0.0,0.0,0.0,0.0,0.0,1680000.0
239975,258795,casa en uno de los desarrollos con mayor plusv...,<p>el desarrollo cuenta con barda perimetral c...,Casa,NaN,Querétaro,Querétaro,NaN,3.0,2.0,...,127625.0,20.649712,-100.349840,2014-04-03 00:00:00,0.0,0.0,0.0,1.0,0.0,1275000.0


/home/administrador/facu/metnum-tp3/tp3/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in log
  """Entry point for launching an IPython kernel.


nan

In [3]:
data = A[['antiguedad', 'metroscubiertos', 'metrostotales']].fillna(0)
b = A[['precio']].fillna(0)
print(data)


data['antiguedad'] = data['antiguedad'] / (data['antiguedad'] + 1)
data['metroscubiertos'] = data['metroscubiertos']
data['metrostotales'] = data['metrostotales']**2

data = data.to_numpy()
b = b.to_numpy().flatten()

In [ ]:
utils.kfold(data, b, 3, utils.RMSLE)

In [29]:
def algo(data, phis, props, pred):
    data = data[props + [pred]].fillna(0)
    b = data[pred]
    data = data[props]
    for i in range(len(props)):
        data[props[i]] = phis[i](data[props[i]])
    data = data.to_numpy()
    b = b.to_numpy().flatten()
    return utils.kfold(data, b, 3, utils.RMSLE)

props = ['antiguedad', 'metroscubiertos', 'metrostotales']
phis = [lambda x : 1/(x+1), lambda x:x, lambda x: x**2]
pred = 'precio'

algo(A, phis, props, pred)
    

0.46448919162887864

In [14]:
blockSize = int(data.shape[0] / 3)
i = 0

limite_1 = blockSize*i
limite_2 = min(data.shape[0], blockSize*(i+1))
print("Rangos train: 0-%d y %d-%d"% (limite_1, limite_2, data.shape[0]))
print("Rangos validation: %d-%d"% (limite_1, limite_2))

A_train, b_train = np.concatenate((data[0:limite_1], data[limite_2:])), np.concatenate((b[0:limite_1], b[limite_2:]))
A_val, b_val = data[limite_1:limite_2], b[limite_1:limite_2]

lr = metnum.LinearRegression()
lr.fit(A_train, b_train)
b_predict = lr.predict(A_val).flatten()

print("Real")
print(b_val)
print("Prediccion")
print(b_predict)

res = utils.RMSE(b_predict, b_val)
print("RMSE")
print(res)

res2 = utils.RMSLE(b_predict, b_val)
print("RMSLE")
print(res2)

Rangos train: 0-0 y 10-30
Rangos validation: 0-10
Real
[ 1150000.  5300000.  2600000. 10707900.   525000.   990000.  3800000.
   980000.   570000.   700000.]
Prediccion
[ 589423.08870827 8090240.08953054  746887.58467359  531248.27661411
  531979.49562422 1025295.49436495 1276606.36978176  306740.14811698
  541246.91769707  382496.01080973]
RMSE
3493160.1223388743
RMSLE
1.187893314921118


In [15]:
from sklearn.linear_model import LinearRegression
reg = LinearRegression().fit(A_train, b_train)
b_predict = reg.predict(A_val)
print("Real")
print(b_val)
print("Prediccion")
print(b_predict)

Real
[ 1150000.  5300000.  2600000. 10707900.   525000.   990000.  3800000.
   980000.   570000.   700000.]
Prediccion
[1094271.70821964 7876994.54339528  983584.17896899 1034741.1107042
 1051035.09045045 1253743.25652615 1526617.5008357   842504.7662581
 1041964.31489644  978273.4077479 ]


In [16]:
utils.kfold(data, b, 3, utils.RMSE)

2394498.761106026

In [4]:
utils.kfold(data, b, 3, utils.RMSLE)

0.492814112923218